In [1]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import lightning as L
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
IMAGE_SIZE = 32
mean, std = [0.4914], [0.247]
# source: https://pytorch.org/vision/stable/transforms.html
transforms_train = transforms.Compose([
                                    transforms.Grayscale(num_output_channels=1),
                                    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean, std)])


transforms_test = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                      transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean, std)])


class ClassificationData(L.LightningDataModule):

    def __init__(self, data_dir="../data-no-noise-no-silence", batch_size=128):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def prepare_data(self):
        pass

    def setup(self, stage=None):
        if stage == "fit":
            self.train_dataset = ImageFolder(
                root=f"{self.data_dir}/train", transform=transforms_train
            )
            self.val_dataset = ImageFolder(
                root=f"{self.data_dir}/val", transform=transforms_test
            )
        if stage == "test":
            self.test_dataset = ImageFolder(
                root=f"{self.data_dir}/test", transform=transforms_test
            )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=31,
            persistent_workers=True,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=31,
            persistent_workers=True,
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=31,
            persistent_workers=True,
        )

In [ ]:
from densenet_utils import DenseNetLit
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import MLFlowLogger

torch.set_float32_matmul_precision('medium')

In [5]:
data = ClassificationData(data_dir="../data-no-noise-no-silence", batch_size=512)
hyperparameters = {
    "depth": 40,
    "first_output": 16,
    "growth_rate": 12,
    "dropout": 0.2,
}
model = DenseNetLit(hyperparameters=hyperparameters)
logger = MLFlowLogger(experiment_name="DenseNet", save_dir="mlruns")
trainer = L.Trainer(max_epochs=150, logger=logger, callbacks=
                    [ModelCheckpoint(monitor="val_f1_macro", mode="max", dirpath="checkpoints/densenet", filename="{epoch:02d}-{val_f1_macro:.3f}-{val_accuracy:.3f}")],
                    precision="16-mixed", 
                    num_sanity_val_steps=0)
trainer.fit(model, datamodule=data)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | DenseNet         | 1.0 M  | train
1 | loss_fn       | CrossEntropyLoss | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | val_metrics   | MetricCollection | 0      | train
4 | test_metrics  | MetricCollection | 0      | train
-----------------------------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.080     Total estimated model params size (MB)
256       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=150` reached.


In [13]:
# check on a test set
best_model = DenseNetLit.load_from_checkpoint("checkpoints/densenet/epoch=143-val_f1_macro=0.930-val_accuracy=0.960.ckpt", hyperparameters=hyperparameters)
trainer.test(best_model, datamodule=data)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.9520183205604553
       test_auroc            0.996806263923645
      test_f1_macro         0.9147834777832031
     test_precision         0.9266074895858765
       test_recall          0.9039677381515503
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_accuracy': 0.9520183205604553,
  'test_auroc': 0.996806263923645,
  'test_f1_macro': 0.9147834777832031,
  'test_precision': 0.9266074895858765,
  'test_recall': 0.9039677381515503}]

# Fit once again, but on full dataset

It means that I add "silence" labels to datasets.

In [5]:
data = ClassificationData(data_dir="../data-no-noise-no-silence", batch_size=512)
hyperparameters = {
    "depth": 40,
    "first_output": 16,
    "growth_rate": 12,
    "dropout": 0.2,
}
model = DenseNetLit(hyperparameters=hyperparameters)
logger = MLFlowLogger(experiment_name="DenseNet", save_dir="mlruns")
trainer = L.Trainer(max_epochs=150, logger=logger, callbacks=
                    [ModelCheckpoint(monitor="val_f1_macro", mode="max", dirpath="checkpoints/densenet", filename="full-{epoch:02d}-{val_f1_macro:.3f}-{val_accuracy:.3f}")],
                    precision="16-mixed", 
                    num_sanity_val_steps=0)
trainer.fit(model, datamodule=data)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\SPCX\Desktop\github-repositories\dl-transformers\.venv\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\SPCX\Desktop\github-repositories\dl-transformers\experimental\checkpoints\densenet exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | DenseNet         | 1.0 M  | train
1 | loss_fn       | CrossEntropyLoss | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | val_metrics   | MetricCollection | 0      | train
4 | test_metrics  | MetricCollection | 0      | train
-----------------------------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=150` reached.


In [6]:
best_model = DenseNetLit.load_from_checkpoint("checkpoints/densenet/full-epoch=132-val_f1_macro=0.928-val_accuracy=0.957.ckpt", hyperparameters=hyperparameters)
trainer.test(best_model, datamodule=data)
None

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.9479014873504639
       test_auroc           0.9969915747642517
      test_f1_macro         0.9128718376159668
     test_precision         0.9253436326980591
       test_recall          0.9017865061759949
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
